In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ZeroPadding2D,GlobalAveragePooling2D,Dense,Conv2D, Convolution2D, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras import Input
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split 
from keras import applications
from keras.applications.vgg16 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.models import Model
import keras
from keras_vggface.vggface import VGGFace

import pandas as pd
import tensorflow as tf
import numpy as np
import cv2
import os

# import the Google AutoML client library
from google.cloud import automl_v1beta1 as automl

from google.cloud import storage
import gc
from sys import getsizeof

## Model Training

In [ ]:
labels=list()
data=list()
def load_data(bucket_name):
    
    bucket = storage.Client().get_bucket(bucket_name)
    for item in range(50):
        path = 'Faces/Augmentation Data/Symptoms/'+str(item)+'_symptoms.jpg'
        blob = bucket.get_blob(path)
        img = np.array(cv2.imdecode(np.asarray(bytearray(blob.download_as_string()),dtype=np.uint8),3))
        data.append(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
        labels.append(1)
    for item in range(50):
        path = 'Faces/Augmentation Data/Normal/'+str(item)+'_normal.jpg'
        blob = bucket.get_blob(path)
        img = np.array(cv2.imdecode(np.asarray(bytearray(blob.download_as_string()),dtype=np.uint8),3))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        data.append(img)
        labels.append(0)
        
    return np.asarray(data),np.asarray(labels)

In [ ]:
def visualize(ori,flip,flip2):
    fig = plt.figure()
    plt.subplot(1,3,1)
    plt.title('Random Brightness')
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.imshow(ori.astype(np.uint8))
    
    plt.subplot(1,3,2)
    plt.title('Random Noise')
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.imshow(flip.astype(np.uint8))
    
    plt.subplot(1,3,3)
    plt.title('Random Hue')
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.imshow(flip2.astype(np.uint8))

In [ ]:
visualize(data[5],data[6],data[7])

In [ ]:
X_train, X_test, y_train,y_test = train_test_split(data,labels,test_size=0.2,random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.15,random_state=42)

In [ ]:
X_train_cnn = X_train/255.
X_test_cnn = X_test/255.
X_valid_cnn = X_valid/255.

In [ ]:
conv_base = VGGFace(weights='vggface',
                        include_top=False,
                        input_shape=(200,200,3))
conv_base.trainable = False

model = Sequential()

model.add(conv_base)
model.add(Conv2D(64, (3, 3), activation='relu',padding="Same"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu',padding="Same"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model_trained = model.fit(X_train_cnn, y_train, epochs=5, 
                  verbose=2, validation_data=(X_test_cnn, y_test))

## Explainable AI Setup Google Platform

In [ ]:
random_baseline = np.random.rand(200,200,3)

explanation_metadata = {
    "inputs": {
      "data": {
        "input_tensor_name": "vggface_vgg16_input:0",
        "modality": "image",
        "input_baselines": [random_baseline.tolist()]
      }
    },
    "outputs": {
      "probability": {
        "output_tensor_name": "dense_15/Softmax:0"
      }
    },
  "framework": "tensorflow"
  }


In [ ]:
import json
with open('explanation_metadata.json', 'w') as output_file:
    json.dump(explanation_metadata, output_file)

In [ ]:
!gsutil cp explanation_metadata.json $export_path


In [ ]:
export_path = 'your_path'
MODEL='face'

In [ ]:
!gcloud ai-platform models create $MODEL --enable-logging --regions=us-central1

## Integrated Gradients

In [ ]:
IG_VERSION = 'v_ig'

In [ ]:
!gcloud beta ai-platform versions create $IG_VERSION \
--model $MODEL \
--origin $export_path \
--runtime-version 1.15 \
--framework TENSORFLOW \
--python-version 3.7 \
--machine-type n1-standard-4 \
--explanation-method integrated-gradients \
--num-integral-steps 40

In [ ]:
!gcloud ai-platform versions describe $IG_VERSION --model $MODEL

In [ ]:
instances=list()
data=list()
def load_data(bucket_name):
    
    bucket = storage.Client().get_bucket(bucket_name)
    for item in range(1):
        path = 'Faces/Augmentation Data/Symptoms/'+str(item)+'_symptoms.jpg'
        blob = bucket.get_blob(path)
        img = np.array(cv2.imdecode(np.asarray(bytearray(blob.download_as_string()),dtype=np.uint8),3))
        #data.append()
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        #img = base64.b64encode(img).decode('utf-8')
        img = img.tolist()
        instances.append({'conv2d_input': img})

        
    for item in range(1):
        path = 'Faces/Augmentation Data/Normal/'+str(item)+'_normal.jpg'
        blob = bucket.get_blob(path)
        img = np.array(cv2.imdecode(np.asarray(bytearray(blob.download_as_string()),dtype=np.uint8),3))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        #img = base64.b64encode(img).decode('utf-8')
        img = img.tolist()
        instances.append({'conv2d_input': img})
        
   
        
    return instances

In [ ]:
instances = load_data('bucket_name')

In [ ]:
instances = []
bucket = storage.Client().get_bucket('bucket_name')
path = 'ck.jpg'
blob = bucket.get_blob(path)
img = np.array(cv2.imdecode(np.asarray(bytearray(blob.download_as_string()),dtype=np.uint8),3))
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = base64.b64encode(img).decode('utf-8')
instances.append({'conv2d_input': [{'b64':img}]})

In [ ]:
def predict_json(project, model, instances, version=None):

    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().explain(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response

In [ ]:
test_filenames = []
# path = 'faces2/symptoms/'+str(2593)+'_symptoms.jpg'
# test_filenames.append(path)
    

# path = 'faces2/normal/'+str(2188)+'_normal.jpg'
# test_filenames.append(path)

for i in range(50):
    
    
    path = 'faces2/symptoms/'+str(i)+'_symptoms.jpg'
    test_filenames.append(path)
    
    path = 'faces2/normal/'+str(i)+'_normal.jpg'
    test_filenames.append(path)

In [ ]:
instances = []

for i in test_filenames:
    with open(i,'rb') as img:
        b64str = base64.b64encode(img.read()).decode('utf-8')
        instances.append({'conv2d_input': [{'b64':b64str}]})

In [ ]:
ig_response = predict_json('ambient-depth-287712', MODEL, instances, IG_VERSION)

In [ ]:
from io import BytesIO
import matplotlib.image as mpimg
import base64
classes = {0:'normal',1:'symptoms'}
# Note: change the `ig_response` variable below if you didn't deploy an IG model
for i,val in enumerate(ig_response['explanations']):
    class_name = classes[val['attributions_by_label'][0]['label_index']]
    confidence_score = str(round(val['attributions_by_label'][0]['example_score'] * 100, 3)) + '%'
    print('Predicted class: ' + str(class_name) + '\n' + 'Confidence score: ' + confidence_score)
    
    img = instances[i]['conv2d_input'][0]['b64']
    im = BytesIO(base64.b64decode(img))
    i = mpimg.imread(im, format='JPG')
    plt.imshow(i, interpolation='nearest')
    plt.show()

In [ ]:
import io

for idx, face in enumerate(ig_response['explanations']):
    predicted_face = classes[face['attributions_by_label'][0]['label_index']]
    confidence = face['attributions_by_label'][0]['example_score']
    print('Predicted face: ', predicted_face)
    b64str = face['attributions_by_label'][0]['attributions']['data']['b64_jpeg']
    i = base64.b64decode(b64str)
    i = io.BytesIO(i)
    i = mpimg.imread(i, format='JPG')

    plt.imshow(i, interpolation='nearest')
    plt.show()

In [ ]:
for i,val in enumerate(ig_response['explanations']):
    baseline_score = val['attributions_by_label'][0]['baseline_score']
    predicted_score = val['attributions_by_label'][0]['example_score']
    print('Baseline score: ', baseline_score) 
    print('Predicted score: ', predicted_score)
    print('Predicted - Baseline: ', predicted_score - baseline_score, '\n')

In [ ]:
rand_test_img = PIL.Image.fromarray((random_baseline * 255).astype('uint8'))
buffer = BytesIO()
rand_test_img.save(buffer, format="BMP")
new_image_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

# Preview it
plt.imshow(rand_test_img)
# Save the image to a variable in the format our model is expecting
sanity_check_img = {'conv2d_input': [{'b64': new_image_string}]}

In [ ]:
sanity_check_resp = predict_json('ambient-depth-287712', MODEL, sanity_check_img, IG_VERSION)

In [ ]:
sanity_check_img = base64.b64decode(sanity_check_resp['explanations'][0]['attributions_by_label'][0]['attributions']['data']['b64_jpeg'])
sanity_check_img = io.BytesIO(sanity_check_img)
sanity_check_img = mpimg.imread(sanity_check_img, format='JPG')

plt.imshow(sanity_check_img, interpolation='nearest')
plt.show()

In [ ]:
baseline_score = sanity_check_resp['explanations'][0]['attributions_by_label'][0]['baseline_score']
example_score = sanity_check_resp['explanations'][0]['attributions_by_label'][0]['example_score']

print(abs(baseline_score - example_score))

## XRAI

In [ ]:
explanation_metadata = {
    "inputs": {
      "data": {
        "input_tensor_name": "vggface_vgg16_input:0",
        "modality": "image",
        "visualization": {
        "type": "Pixels", # Can also use "pixels"
        "polarity": "negative",
        "clip_below_percentile": 0,
        "clip_above_percentile": 100,
        "color_map": "viridis",
        "overlay_type": "grayscale"
        },
        "input_baselines": [random_baseline.tolist()]
      }
    },
    "outputs": {
      "probability": {
        "output_tensor_name": "dense_15/Softmax:0"
      }
    },
  "framework": "tensorflow"
  }

In [ ]:
import json
with open('explanation_metadata.json', 'w') as output_file:
    json.dump(explanation_metadata, output_file)

In [ ]:
!gsutil cp explanation_metadata.json $export_path

In [ ]:
XRAI_VERSION = 'v_xrai'

In [ ]:
!gcloud beta ai-platform versions create $XRAI_VERSION \
--model $MODEL \
--origin $export_path \
--runtime-version 1.15 \
--framework TENSORFLOW \
--python-version 3.7 \
--machine-type n1-standard-4 \
--explanation-method xrai \
--num-integral-steps 25

In [ ]:
xrai_response = predict_json('ambient-depth-287712', MODEL, instances, XRAI_VERSION)

In [ ]:
from io import BytesIO
import matplotlib.image as mpimg
import base64
classes = {0:'normal',1:'symptoms'}
# Note: change the `ig_response` variable below if you didn't deploy an IG model
for i,val in enumerate(xrai_response['explanations']):
    class_name = classes[val['attributions_by_label'][0]['label_index']]
    confidence_score = str(round(val['attributions_by_label'][0]['example_score'] * 100, 3)) + '%'
    print('Predicted class: ' + str(class_name) + '\n' + 'Confidence score: ' + confidence_score)
    
    img = instances[i]['conv2d_input'][0]['b64']
    im = BytesIO(base64.b64decode(img))
    i = mpimg.imread(im, format='JPG')
    plt.imshow(i, interpolation='nearest')
    plt.show()

In [ ]:
import io

for idx, face in enumerate(xrai_response['explanations']):
    predicted_face = classes[face['attributions_by_label'][0]['label_index']]
    confidence = face['attributions_by_label'][0]['example_score']
    print('Predicted face: ', predicted_face)
    b64str = face['attributions_by_label'][0]['attributions']['data']['b64_jpeg']
    i = base64.b64decode(b64str)
    i = io.BytesIO(i)
    i = mpimg.imread(i, format='JPG')

    plt.imshow(i, interpolation='nearest')
    plt.show()

In [ ]:
for i,val in enumerate(xrai_response['explanations']):
    baseline_score = val['attributions_by_label'][0]['baseline_score']
    predicted_score = val['attributions_by_label'][0]['example_score']
    print('Baseline score: ', baseline_score) 
    print('Predicted score: ', predicted_score)
    print('Predicted - Baseline: ', predicted_score - baseline_score, '\n')

In [ ]:
rand_test_img = PIL.Image.fromarray((random_baseline * 255).astype('uint8'))
buffer = BytesIO()
rand_test_img.save(buffer, format="BMP")
new_image_string = base64.b64encode(buffer.getvalue()).decode("utf-8")

# Preview it
plt.imshow(rand_test_img)
# Save the image to a variable in the format our model is expecting
sanity_check_img = {'conv2d_input': [{'b64': new_image_string}]}

In [ ]:
sanity_check_resp = predict_json('ambient-depth-287712', MODEL, sanity_check_img, XRAI_VERSION)

In [ ]:
sanity_check_img = base64.b64decode(sanity_check_resp['explanations'][0]['attributions_by_label'][0]['attributions']['data']['b64_jpeg'])
sanity_check_img = io.BytesIO(sanity_check_img)
sanity_check_img = mpimg.imread(sanity_check_img, format='JPG')

plt.imshow(sanity_check_img, interpolation='nearest')
plt.show()

In [ ]:
baseline_score = sanity_check_resp['explanations'][0]['attributions_by_label'][0]['baseline_score']
example_score = sanity_check_resp['explanations'][0]['attributions_by_label'][0]['example_score']

print(abs(baseline_score - example_score))

## Different Parameter Setting for Visualization

In [ ]:
explanation_metadata = {
    "inputs": {
      "data": {
        "input_tensor_name": "vggface_vgg16_input:0",
        "modality": "image",
        "visualization": {
        "type": "Outlines", # Can also use "pixels"
        "polarity": "negative",
        "clip_below_percentile": 55,
        "clip_above_percentile": 99.9,
        "color_map": "pink_green",
        "overlay_type": "mask_black"
        },
        "input_baselines": [random_baseline.tolist()]
      }
    },
    "outputs": {
      "probability": {
        "output_tensor_name": "dense_15/Softmax:0"
      }
    },
  "framework": "tensorflow"
  }

In [ ]:
import json
with open('explanation_metadata.json', 'w') as output_file:
    json.dump(explanation_metadata, output_file)

In [ ]:
!gsutil cp explanation_metadata.json $export_path

In [ ]:
IG_VERSION = 'v_ig_parameter1'
! gcloud ai-platform versions delete $IG_VERSION --quiet --model $MODEL

In [ ]:
!gcloud beta ai-platform versions create $IG_VERSION \
--model $MODEL \
--origin $export_path \
--runtime-version 1.15 \
--framework TENSORFLOW \
--python-version 3.7 \
--machine-type n1-standard-4 \
--explanation-method integrated-gradients \
--num-integral-steps 60

In [ ]:
!gcloud ai-platform versions describe $IG_VERSION --model $MODEL

In [ ]:
test_filenames = []
path = 'faces2/symptoms/'+str(1700)+'_symptoms.jpg'
test_filenames.append(path)

In [ ]:
instances = []
import base64
import matplotlib.image as mpimg
import googleapiclient
for i in test_filenames:
    with open(i,'rb') as img:
        b64str = base64.b64encode(img.read()).decode('utf-8')
        instances.append({'conv2d_input': [{'b64':b64str}]})

In [ ]:
ig_response = predict_json('ambient-depth-287712', MODEL, instances, IG_VERSION)

In [ ]:
from io import BytesIO
import matplotlib.image as mpimg
import base64
classes = {0:'normal',1:'symptoms'}
# Note: change the `ig_response` variable below if you didn't deploy an IG model
for i,val in enumerate(ig_response['explanations']):
    class_name = classes[val['attributions_by_label'][0]['label_index']]
    confidence_score = str(round(val['attributions_by_label'][0]['example_score'] * 100, 3)) + '%'
    print('Predicted class: ' + str(class_name) + '\n' + 'Confidence score: ' + confidence_score)
    
    img = instances[i]['conv2d_input'][0]['b64']
    im = BytesIO(base64.b64decode(img))
    i = mpimg.imread(im, format='JPG')
    plt.imshow(i, interpolation='nearest')
    plt.show()

In [ ]:
import io
classes = {0:'normal',1:'symptoms'}
for idx, face in enumerate(ig_response['explanations']):
    predicted_face = classes[face['attributions_by_label'][0]['label_index']]
    confidence = face['attributions_by_label'][0]['example_score']
    print('Predicted face: ', predicted_face)
    b64str = face['attributions_by_label'][0]['attributions']['data']['b64_jpeg']
    i = base64.b64decode(b64str)
    i = io.BytesIO(i)
    i = mpimg.imread(i, format='JPG')

    plt.imshow(i, interpolation='nearest')
    plt.axis('off')
    plt.show()

In [ ]:
for i,val in enumerate(ig_response['explanations']):
    baseline_score = val['attributions_by_label'][0]['baseline_score']
    predicted_score = val['attributions_by_label'][0]['example_score']
    print('Baseline score: ', baseline_score) 
    print('Predicted score: ', predicted_score)
    print('Predicted - Baseline: ', predicted_score - baseline_score, '\n')
    approx_error = val['attributions_by_label'][0]['approx_error']
    print('Approximate error: ',approx_error)